In [1]:
import torch
import torch.nn as nn
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import binned_statistic

from models import CFM
from models import Classifier
from gaussian_toy import GaussianToy
from plots import plot_naive_unfold, plot_reweighted_distribution, plot_prior_unfold, SetStyle
SetStyle()

In [2]:
import logging
import sys
for h in logging.root.handlers[:]:
    logging.root.removeHandler(h)
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="%(asctime)s %(levelname)s %(message)s",
)

# Create Gaussian toy example. Define six datasets:
1. Reco-level simulation
2. Gen-level simulation
3. Background simulation
4. Reco-level data
5. Gen-level data
6. Background data

In [ ]:
device = 'cuda'
data_params = { "n_dim": 1,
                "n_mc": 1_000_000,
                "mc_mu": 0,
                "mc_sigma": 1,
                "n_data": 1_000_000,
                "data_mu": 0.2,
                "data_sigma": 0.8,
                "detector_mu": 0,
                "detector_sigma": 0.5,
                "n_background": 100_000,
                "background_mu": 0,
                "background_sigma": 1.2,
                "mc_rec_cut": True,
                "mc_gen_cut": True,
                "data_rec_cut": True,
                "data_gen_cut": True ,
                "efficiency": 0.1,
                "acceptance": 0.1,
                "empty_value": -5.0,
}

In [ ]:
ToyModel = GaussianToy(data_params)

# Define background subtraction CFM

In [ ]:
bkg_mc = ToyModel.mc_background_rec
data_rec = ToyModel.data_rec

In [ ]:
generator_params = {"hidden_layers": 4,
                   "internal_size": 64,
                   "lr": 1.e-5,
                   "n_epochs" : 100,
                   "batch_size" : 128,
                   "batch_size_sample": 2000}

In [ ]:
signal_generator = CFM(data_params['n_dim'], 0, generator_params)

In [ ]:
weights = torch.cat([torch.ones_like(data_rec[ToyModel.data_rec_mask.bool(),0]),
                     -torch.ones_like(bkg_mc[:,0])])
data = torch.cat([data_rec[ToyModel.data_rec_mask.bool()], bkg_mc], 0)
signal_generator.train(data, weights=weights)

# Generate the signal and generate empty events at reco level

In [ ]:
num_data_reco = ToyModel.data_signal_rec[:,0][ToyModel.data_rec_mask[:data_params["n_data"]].bool()].size(0)
generated_signal = signal_generator.evaluate(num_evts = num_data_reco,device = device) #N*(1-delta)

In [ ]:
y_true = plt.hist(ToyModel.data_signal_rec[:,0][ToyModel.data_rec_mask[:data_params["n_data"]].bool()].cpu().detach().numpy(),bins = 60, range = [-5.5,4],label = "True signal", histtype='step')
y_gen = plt.hist(generated_signal[:,0].cpu().detach().numpy(),bins = 60, range = [-5.5,4],label = "Generated signal", histtype='step')
plt.legend()

In [ ]:
num_data_empty = num_data_reco*data_params['acceptance']/(1.0 - data_params['acceptance']) #N*(1-delta)*epsilon/(1-epsilon)
#Add the empty events to the generated signals
generated_signal = torch.cat([generated_signal,data_params["empty_value"]*torch.ones_like(generated_signal[:int(num_data_empty)])])
signal_mask = generated_signal[:,0] != data_params["empty_value"]

In [ ]:
print(f"Number of expected signal events in the data {generated_signal.size(0)}")

# Train Acceptance classifier

In [ ]:
acceptance_true = ToyModel.mc_rec[(ToyModel.mc_rec_mask.bool()) & (ToyModel.mc_gen_mask.bool())]
acceptance_false = ToyModel.mc_rec[(ToyModel.mc_rec_mask.bool()) & ~(ToyModel.mc_gen_mask.bool())]

In [ ]:
acceptance_classifier_params = { "hidden_layers": 4,
                                 "internal_size": 64,
                                 "lr": 3.e-5,
                                 "n_epochs" : 30,
                                 "batch_size" : 128,
                                 "batch_size_sample": 2000
}

In [ ]:
acceptance_classifier = Classifier(dims_in=1, params=acceptance_classifier_params,model_name="acceptance classifier")

In [ ]:
acceptance_classifier.train_classifier(acceptance_true, acceptance_false, balanced=False)

# Train efficiency classifier

In [ ]:
efficiency_classifier_params = { "hidden_layers": 4,
                                 "internal_size": 64,
                                 "lr": 1.e-4,
                                 "n_epochs" : 30,
                                 "batch_size" : 128,
                                 "batch_size_sample": 2000
}

In [ ]:
efficiency_true = ToyModel.mc_gen[(ToyModel.mc_rec_mask.bool()) & (ToyModel.mc_gen_mask.bool())]
efficiency_false = ToyModel.mc_gen[~(ToyModel.mc_rec_mask.bool()) & (ToyModel.mc_gen_mask.bool())]

In [ ]:
efficiency_classifier = Classifier(dims_in = 1, params = efficiency_classifier_params,model_name="efficiency classifier")

In [ ]:
efficiency_classifier.train_classifier(efficiency_true, efficiency_false, balanced=False)

# Let's train the detector response flow p(reco|gen) and the initial p(gen) flows

In [ ]:
gen_generator = CFM(dims_x = data_params['n_dim'], dims_c = 0,params = generator_params)

In [ ]:
gen_generator.train(ToyModel.mc_gen[ToyModel.mc_gen_mask.bool()],weights = torch.ones_like(ToyModel.mc_gen[ToyModel.mc_gen_mask.bool()][:,0]))

In [ ]:
y_true = plt.hist(ToyModel.mc_gen[:,0][ToyModel.mc_gen_mask.bool()].cpu().detach().numpy(),
                  bins = 60, range = [-5.5,4],label = "MC gen signal", histtype='step')
y_gen = plt.hist(gen_generator.evaluate(num_evts = ToyModel.mc_gen[ToyModel.mc_gen_mask.bool()].size(0),device=device).cpu().detach().numpy(),
                 bins = 60, range = [-5.5,4],label = "Generated gen signal", histtype='step')
plt.legend()

In [ ]:
detector_generator = CFM(dims_x = data_params['n_dim'], dims_c = data_params['n_dim'],params = generator_params)

In [ ]:
detector_generator.train(ToyModel.mc_rec[ToyModel.mc_rec_mask.bool()],
                         weights = torch.ones_like(ToyModel.mc_rec[ToyModel.mc_rec_mask.bool()][:,0]), 
                         data_c = ToyModel.mc_gen[ToyModel.mc_rec_mask.bool()])

In [ ]:
y_true = plt.hist(ToyModel.mc_rec[:,0][ToyModel.mc_rec_mask.bool()].cpu().detach().numpy(),
                  bins = 60, range = [-5.5,4],label = "MC reco signal", histtype='step')
y_gen = plt.hist(detector_generator.evaluate(data_c = ToyModel.mc_gen[ToyModel.mc_rec_mask.bool()]).cpu().detach().numpy(),
                 bins = 60, range = [-5.5,4],label = "Generated MC reco signal", histtype='step')
plt.legend()

# Start the unfolding!

In [ ]:
def sample_reco(nevts,empty_fraction, efficiency_classifier,detector_model,gen_model):
    ''' Generates h(reco|gen) samples by sampling from: h(reco|gen) = c(gen) + (1-c(gen))*p(reco|gen)*p(gen) '''
    gen_events = gen_model.evaluate(num_evts = nevts,device=device)
    gen_mask = ToyModel.apply_efficiency_acceptance_effects(gen_events,empty_fraction)
    #By definition generated events are within acceptance
    gen_events = gen_events[gen_mask.bool()] 
    efficiency = efficiency_classifier.evaluate(gen_events, return_weights=False)
    sample_efficiency = torch.bernoulli(efficiency)


    num_data_empty = nevts*data_params['efficiency']/(1.0 - data_params['efficiency']) #N*(1-delta)*epsilon/(1-epsilon)
    gen_events = torch.cat([gen_events,
                            data_params["empty_value"] * torch.ones_like(gen_events[: int(num_data_empty)])])
    gen_mask = gen_events[:,0] != data_params["empty_value"] 
    
    #By definition, events not passing gen should pass reco
    sample_efficiency = torch.cat(
        [sample_efficiency,
         torch.ones_like(sample_efficiency[:int(num_data_empty)])]
        )
    
    reco_events = detector_model.evaluate(data_c=gen_events)
    reco_events[~sample_efficiency.bool()] = data_params["empty_value"]*torch.ones_like(gen_events[~sample_efficiency.bool()])
    return  reco_events, gen_events, gen_mask, sample_efficiency

In [ ]:
iterations = 5
unfold_generator =  CFM(dims_x = data_params['n_dim'], dims_c = data_params['n_dim'],params = generator_params)
for i in range(iterations):
    print(f"Running iteration {i}")
    reco_train, gen_train, gen_mask, reco_mask = sample_reco(data_params['n_mc'], 
                                                             data_params['efficiency'],
                                                             efficiency_classifier,
                                                             detector_generator,
                                                             gen_generator)
    unfold_generator.train(gen_train[gen_mask.bool()], 
                           weights = torch.ones_like(gen_train[gen_mask.bool()][:,0]), 
                           data_c = reco_train[gen_mask.bool()])
    
    #Update the acceptance model
    acceptance_classifier.train_classifier(
                reco_train[(reco_mask.bool()) & (gen_mask.bool())],
                reco_train[(reco_mask.bool()) & ~(gen_mask.bool())], balanced=False
            )
    acceptance = torch.cat([acceptance_classifier.evaluate(generated_signal[signal_mask.bool()], return_weights=False),
                            torch.ones_like(generated_signal[~signal_mask.bool()][:,0])],-1)    
    acceptance_mask = torch.bernoulli(acceptance)
    
    #Generate unfolded events
    unfolded = unfold_generator.evaluate(data_c=generated_signal)
    
    #Train generator model after applying the acceptance model
    gen_generator.train(unfolded[acceptance_mask.bool()], weights=torch.ones_like(unfolded[:, 0][acceptance_mask.bool()]))

    fig, axes = plt.subplots()
    axes.hist(unfolded[:,0].cpu().detach().numpy(), bins=60, histtype="step", range=[-5.5,4],label="Gen. Unfolded",density=True)
    axes.hist(ToyModel.mc_gen[:,0][ToyModel.mc_gen_mask.bool()].cpu().detach().numpy(),bins=60, histtype='step', range=[-5.5,4],label="Gen. MC",density=True)
    axes.hist(ToyModel.data_gen[:,0][ToyModel.data_gen_mask.bool()].cpu().detach().numpy(), 
              bins=60, range=[-5.5,4], histtype="step", label="Gen. Truth",density=True)
    plt.legend()  # Display the legend
    plt.show()

In [ ]:
with PdfPages(f"Plots/final_generative_unfolding.pdf") as out:
    plot_naive_unfold(out, 
                      ToyModel.data_gen[:, 0][ToyModel.data_gen_mask.bool()].cpu().detach().numpy(),
                      ToyModel.data_rec[:, 0][ToyModel.data_rec_mask.bool()].cpu().detach().numpy(),
                      unfolded[:, 0].cpu().detach().numpy(),
                      range=[-3, 4], name="x_1")